https://mp.weixin.qq.com/s/jQGNWEdxH0meVwk-YDN4gw <br>
https://blog.csdn.net/baymax_007/article/details/83931698

In [12]:
import tensorflow as tf
import numpy as np

In [2]:
embedding = tf.constant(
    [[0.21,0.41,0.51,0.11],
    [0.22,0.42,0.52,0.12],
    [0.23,0.43,0.53,0.13],
    [0.24,0.44,0.54,0.14]],dtype=tf.float32
)
feature_batch = tf.constant([2,3,1,0])
get_embedding1 = tf.nn.embedding_lookup(embedding,feature_batch)

In [3]:
import pandas as pd

In [4]:
TRAIN_FILE = "../../dataset/deepfm/train.csv"
TEST_FILE = "../../dataset/deepfm/test.csv"
NUMERIC_COLS = [
"ps_reg_01", "ps_reg_02", "ps_reg_03",
"ps_car_12", "ps_car_13", "ps_car_14", "ps_car_15"
]

IGNORE_COLS = [
"id", "target",
"ps_calc_01", "ps_calc_02", "ps_calc_03", "ps_calc_04",
"ps_calc_05", "ps_calc_06", "ps_calc_07", "ps_calc_08",
"ps_calc_09", "ps_calc_10", "ps_calc_11", "ps_calc_12",
"ps_calc_13", "ps_calc_14",
"ps_calc_15_bin", "ps_calc_16_bin", "ps_calc_17_bin",
"ps_calc_18_bin", "ps_calc_19_bin", "ps_calc_20_bin"
]
dfTrain = pd.read_csv(TRAIN_FILE)
dfTest = pd.read_csv(TEST_FILE)
print(dfTrain.head(10))

   id  target  ps_ind_01  ps_ind_02_cat  ps_ind_03  ps_ind_04_cat  \
0   7       0          2              2          5              1   
1   9       0          1              1          7              0   
2  13       0          5              4          9              1   
3  16       0          0              1          2              0   
4  17       0          0              2          0              1   
5  19       0          5              1          4              0   
6  20       0          2              1          3              1   
7  22       0          5              1          4              0   
8  26       0          5              1          3              1   
9  28       1          1              1          2              0   

   ps_ind_05_cat  ps_ind_06_bin  ps_ind_07_bin  ps_ind_08_bin  ...  \
0              0              0              1              0  ...   
1              0              0              0              1  ...   
2              0              

In [5]:
df = pd.concat([dfTrain, dfTest])
feature_dict = {}
total_feature = 0
for col in df.columns:
    if col in IGNORE_COLS:
        continue
    elif col in NUMERIC_COLS:
        feature_dict[col] = total_feature
        total_feature += 1
    else:
        unique_val = df[col].unique()
        feature_dict[col] = dict(zip(unique_val, range(total_feature, len(unique_val) + total_feature)))
        total_feature += len(unique_val)
print(total_feature)
print(feature_dict)

254
{'ps_ind_01': {2: 0, 1: 1, 5: 2, 0: 3, 4: 4, 3: 5, 6: 6, 7: 7}, 'ps_ind_02_cat': {2: 8, 1: 9, 4: 10, 3: 11, -1: 12}, 'ps_ind_03': {5: 13, 7: 14, 9: 15, 2: 16, 0: 17, 4: 18, 3: 19, 1: 20, 11: 21, 6: 22, 8: 23, 10: 24}, 'ps_ind_04_cat': {1: 25, 0: 26, -1: 27}, 'ps_ind_05_cat': {0: 28, 1: 29, 4: 30, 3: 31, 6: 32, 5: 33, -1: 34, 2: 35}, 'ps_ind_06_bin': {0: 36, 1: 37}, 'ps_ind_07_bin': {1: 38, 0: 39}, 'ps_ind_08_bin': {0: 40, 1: 41}, 'ps_ind_09_bin': {0: 42, 1: 43}, 'ps_ind_10_bin': {0: 44, 1: 45}, 'ps_ind_11_bin': {0: 46, 1: 47}, 'ps_ind_12_bin': {0: 48, 1: 49}, 'ps_ind_13_bin': {0: 50, 1: 51}, 'ps_ind_14': {0: 52, 1: 53, 2: 54, 3: 55}, 'ps_ind_15': {11: 56, 3: 57, 12: 58, 8: 59, 9: 60, 6: 61, 13: 62, 4: 63, 10: 64, 5: 65, 7: 66, 2: 67, 0: 68, 1: 69}, 'ps_ind_16_bin': {0: 70, 1: 71}, 'ps_ind_17_bin': {1: 72, 0: 73}, 'ps_ind_18_bin': {0: 74, 1: 75}, 'ps_reg_01': 76, 'ps_reg_02': 77, 'ps_reg_03': 78, 'ps_car_01_cat': {10: 79, 11: 80, 7: 81, 6: 82, 9: 83, 5: 84, 4: 85, 8: 86, 3: 87, 0: 8

In [6]:
# 对训练集进行转化
print(dfTrain.columns)
train_y = dfTrain[['target']].values.tolist()
dfTrain.drop(['target','id'],axis=1,inplace=True)
train_feature_index = dfTrain.copy()
train_feature_value = dfTrain.copy()
for col in train_feature_index.columns:
    if col in IGNORE_COLS:
        train_feature_index.drop(col,axis=1,inplace=True)
        train_feature_value.drop(col,axis=1,inplace=True)
        continue
    elif col in NUMERIC_COLS:
        train_feature_index[col] = feature_dict[col]
    else:
        train_feature_index[col] = train_feature_index[col].map(feature_dict[col])
        train_feature_value[col] = 1

Index(['id', 'target', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15',
       'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01',
       'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat',
       'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14',
       'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin'],


In [7]:
train_feature_index.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,0,8,13,25,28,36,38,40,42,44,...,128,131,133,139,142,246,250,251,252,253
1,1,9,14,26,28,36,39,41,42,44,...,128,132,134,139,143,247,250,251,252,253
2,2,10,15,25,28,36,39,41,42,44,...,128,132,134,139,144,248,250,251,252,253
3,3,9,16,26,28,37,39,40,42,44,...,128,132,135,139,145,248,250,251,252,253
4,3,8,17,25,28,37,39,40,42,44,...,128,132,134,139,146,247,250,251,252,253


In [8]:
train_feature_value.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0.400000,0.883679,0.370810,3.605551
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0.316228,0.618817,0.388716,2.449490
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0.316228,0.641586,0.347275,3.316625
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0.374166,0.542949,0.294958,2.000000
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0.316070,0.565832,0.365103,2.000000


In [9]:
# 对测试集进行转化
test_ids = dfTest['id'].values.tolist()
dfTest.drop(['id'], axis=1, inplace=True)

test_feature_index = dfTest.copy()
test_feature_value = dfTest.copy()

for col in test_feature_index.columns:
    if col in IGNORE_COLS:
        test_feature_index.drop(col, axis=1, inplace=True)
        test_feature_value.drop(col, axis=1, inplace=True)
        continue
    elif col in NUMERIC_COLS:
        test_feature_index[col] = feature_dict[col]
    else:
        test_feature_index[col] = test_feature_index[col].map(feature_dict[col])
        test_feature_value[col] = 1

In [10]:
# 模型参数及输入
'''模型参数'''
dfm_params = {
    "use_fm":True,
    "use_deep":True,
    "embedding_size":8,
    "dropout_fm":[1.0,1.0],
    "deep_layers":[32,32],
    "dropout_deep":[0.5,0.5,0.5],
    "deep_layer_activation":tf.nn.relu,
    "epoch":30,
    "batch_size":1024,
    "learning_rate":0.001,
    "optimizer":"adam",
    "batch_norm":1,
    "batch_norm_decay":0.995,
    "l2_reg":0.01,
    "verbose":True,
    "eval_metric":'gini_norm',
    "random_seed":3
}
dfm_params['feature_size'] = total_feature
dfm_params['field_size'] = len(train_feature_index.columns)

In [11]:
# 开始建立模型
feat_index = tf.placeholder(tf.int32,shape=[None,None],name='feat_index')
feat_value = tf.placeholder(tf.float32,shape=[None,None],name='feat_value')
label = tf.placeholder(tf.float32,shape=[None,1],name='label')

In [ ]:
# 建立weights
weights = dict()
# embeddings
weights['feature_embeddings'] = tf.Variable(tf.random_normal([dfm_params['feature_size'],dfm_params['embedding_size']],0.0,0.01),name='feature_embeddings')
weights['feature_bias'] = tf.Variable(tf.random_normal([dfm_params['feature_size'],1],0.0,1.0),name='feature_bias')
# deep layers
num_layer = len(dfm_params['deep_layers'])
input_size = dfm_params['field_size']*dfm_params['embedding_size']
glorot = np.sqrt(2.0/(input_size + dfm_params['deep_layers'][0]))